In [ ]:
    #imports
    import matplotlib.pyplot as plt
    import numpy as np
    import PIL
    import tensorflow as tf
    import os

    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.models import Sequential

    import logging

    # Set the logging level to suppress TensorFlow warnings
    logging.getLogger('tensorflow').setLevel(logging.ERROR)

    
    #Creation of dataset
    train_dir = 'kaggle/train'
    test_dir = 'kaggle/test'
    img_height = 224
    img_width = 224
    batch_size = 32

    # Create train dataset
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_dir,
        image_size=(img_height, img_width),
        batch_size=batch_size)

    # Create test dataset
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        test_dir,
        image_size=(img_height, img_width),
        batch_size=batch_size)

    #todo1
    class_names = train_ds.class_names
    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

    normalization_layer = layers.Rescaling(1./255)
    normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    image_batch, labels_batch = next(iter(normalized_ds))
    first_image = image_batch[0]
    # Notice the pixel values are now in `[0,1]`.
    print(np.min(first_image), np.max(first_image))

    num_classes = len(class_names)
    data_augmentation = keras.Sequential(
      [
        layers.RandomFlip("horizontal",
                          input_shape=(img_height,
                                      img_width,
                                      3)),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
      ]
    )

    for images, _ in train_ds.take(1):
        for i in range(9):
            augmented_images = data_augmentation(images)

    model = Sequential([
      data_augmentation,
      layers.Rescaling(1./255),
      layers.Conv2D(16, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(32, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Conv2D(64, 3, padding='same', activation='relu'),
      layers.MaxPooling2D(),
      layers.Dropout(0.2),
      layers.Flatten(),
      layers.Dense(128, activation='relu'),
      layers.Dense(num_classes, activation='softmax')
    ])


    # model.compile(optimizer='adam',
    #               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    #               metrics=['accuracy'])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    model.summary()

    epochs=8
    history = model.fit(
      train_ds,
      validation_data=val_ds,
      epochs=epochs
    )

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']
    print(acc)
    print(val_acc)
    print(loss)
    print(val_loss)

    !mkdir -p saved_model
    saved_model_dir = 'saved_model_v6/my_model_to_metadata'
    model.save(saved_model_dir)

In [1]:
import tensorflow as tf

# Convert the model
saved_model_dir = "C:\\Users\\skaluzinski\\saved_model_v6\\my_model_to_metadata"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir) 
tflite_model = converter.convert()

# Save the model.
with open('model_saved_with_lavels.tflite', 'wb') as f:
  f.write(tflite_model)
print("Saved model to dir")

Saved model to dir
